In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')
import pandas as pd
import glob
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jillian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jillian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df_text=pd.read_pickle('primary_df.pkl')
df_text

,raw_text,sect
source,,
Dead_Suns_1,SUNRISE MAIDEN\n\n\nTIER 3\n\nSanjaval Vagabon...,SUNRISE MAIDEN
Dead_Suns_1,SUNRISE MAIDEN\n\n\nTIER 3\n\nSanjaval Vagabon...,TIER 3\n\nSanjaval Vagabond\nMedium explorer\n...
Dead_Suns_1,SUNRISE MAIDEN\n\n\nTIER 3\n\nSanjaval Vagabon...,SUNRISE MAIDEN MAP KEY\n1. Bridge\na. Pilot’s ...
Dead_Suns_1,SUNRISE MAIDEN\n\n\nTIER 3\n\nSanjaval Vagabon...,"\nFORTUNE IN THE STARS\n\nTake your station, c..."
Dead_Suns_1,SUNRISE MAIDEN\n\n\nTIER 3\n\nSanjaval Vagabon...,The Starfinder Adventure Path is designed to p...
...,...,...
Dead_Suns_2,THAUMTECH\nCAIRNCARVER\n\nTIER 2\n\nSmall ligh...,8
Dead_Suns_2,THAUMTECH\nCAIRNCARVER\n\nTIER 2\n\nSmall ligh...,"4,800"
Dead_Suns_2,THAUMTECH\nCAIRNCARVER\n\nTIER 2\n\nSmall ligh...,\nBulldoze (Ex) When a yaruk uses its trample ...


In [3]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, 
    remove punctuation and remove words containing numbers, remove line breaks.'''
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    return text

round1 = lambda x: clean_text(x)

In [4]:
data_clean=pd.DataFrame(df_text.sect.apply(round1))
data_clean

,sect
source,
Dead_Suns_1,sunrise maiden
Dead_Suns_1,tier sanjaval vagabond medium explorer spee...
Dead_Suns_1,sunrise maiden map key bridge a pilot s st...
Dead_Suns_1,fortune in the stars take your station chec...
Dead_Suns_1,the starfinder adventure path is designed to p...
...,...
Dead_Suns_2,
Dead_Suns_2,
Dead_Suns_2,bulldoze ex when a yaruk uses its trample a...


In [5]:
def preprocess_text(text):
    from nltk.corpus import stopwords
    #Tokenize words while ignoring punctuation
    #letters only, strings longer than 2 letters
    tokeniser = RegexpTokenizer(r'[A-Za-z]{3,}')
    tokens = tokeniser.tokenize(text)
     # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(),pos='v') for token in tokens]
    #ps=PorterStemmer()
    #ports=[ps.stem(token.lower())for token in tokens]
    
    
    # Remove creating and removing stopwords
    stopwords= stopwords.words('english')
    new_stopwords=['statistics','tactics','ecology','bulk','defense','offense','treasure',
                  'mission','pcs','starfinder',''] #special abilities?
    stopwords.extend(new_stopwords)
    keywords= [lemma for lemma in lemmas if lemma not in stopwords]
    #keywords= [port for port in ports if port not in stopwords]
    return keywords

In [6]:
vectorizer = CountVectorizer(analyzer=preprocess_text)
doc_word = vectorizer.fit_transform(data_clean.sect)
doc_word.shape

(6558, 22160)

In [7]:
dtm=pd.DataFrame(doc_word.toarray(),columns=vectorizer.get_feature_names())
dtm.index=data_clean.index
dtm

,aanung,abadar,abadaran,abadarans,abadarcorp,abaddon,aballon,aballonian,abandon,abandonment,...,zoologists,zoom,zoos,zunar,zurnala,zusleggim,zyakama,zybollo,zygotes,zysyk
source,,,,,,,,,,,,,,,,,,,,,
Dead_Suns_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dead_Suns_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dead_Suns_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dead_Suns_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dead_Suns_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dead_Suns_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dead_Suns_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dead_Suns_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
nmf_model = NMF(20)
doc_topic = nmf_model.fit_transform(doc_word)
doc_topic.shape

/home/jillian/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/home/jillian/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


(6558, 20)

In [9]:
topic_word = nmf_model.components_
topic_word.shape

(20, 22160)

In [10]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-15:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['computer',
  'computers',
  'data',
  'access',
  'room',
  'check',
  'module',
  'cultists',
  'hack',
  'control',
  'contain',
  'information',
  'base',
  'find'],
 ['planet',
  'city',
  'worlds',
  'many',
  'pact',
  'one',
  'within',
  'live',
  'world',
  'even',
  'find',
  'species',
  'though',
  'system'],
 ['creature',
  'save',
  'effect',
  'target',
  'throw',
  'round',
  'feet',
  'damage',
  'creatures',
  'take',
  'affect',
  'within',
  'action',
  'mind'],
 ['ship',
  'starship',
  'crew',
  'drift',
  'pilot',
  'vessel',
  'rank',
  'captain',
  'engineer',
  'combat',
  'officer',
  'space',
  'rock',
  'system'],
 ['check',
  'succeed',
  'successful',
  'attempt',
  'one',
  'take',
  'perception',
  'engineer',
  'bonus',
  'must',
  'require',
  'circumstance',
  'skill',
  'diplomacy'],
 ['swarm',
  'hylax',
  'suskillon',
  'components',
  'mind',
  'shirrens',
  'sdf',
  'hive',
  'units',
  'temple',
  'creatures',
  'commit',
  'take',
  'soldie

In [11]:
data_clean1=data_clean.copy()
data_clean1['topic']=doc_topic.argmax(axis=1)

In [12]:
data_clean1.to_pickle('topics.pkl')
data_clean1

,sect,topic
source,,
Dead_Suns_1,sunrise maiden,3
Dead_Suns_1,tier sanjaval vagabond medium explorer spee...,3
Dead_Suns_1,sunrise maiden map key bridge a pilot s st...,13
Dead_Suns_1,fortune in the stars take your station chec...,1
Dead_Suns_1,the starfinder adventure path is designed to p...,1
...,...,...
Dead_Suns_2,,0
Dead_Suns_2,,0
Dead_Suns_2,bulldoze ex when a yaruk uses its trample a...,14


In [13]:
doc_cluster=doc_topic.argmax(axis=1)
pd.Series(doc_cluster).value_counts()

14    1317
16     838
1      551
17     517
0      436
2      408
4      337
9      299
3      252
5      242
11     232
18     218
7      194
19     170
8      164
12     125
6       75
10      71
13      70
15      42
dtype: int64

In [14]:
from sklearn.decomposition import LatentDirichletAllocation

In [15]:
lda_model = LatentDirichletAllocation(n_components=20)
lda_doc_topic = lda_model.fit_transform(doc_word)
lda_doc_topic.shape

(6558, 20)

In [16]:
words = vectorizer.get_feature_names()
ldat = lda_model.components_.argsort(axis=1)[:,-1:-7:-1]
lda_topic_words = [[words[e] for e in l] for l in ldat]
lda_topic_words

[['gray', 'use', 'one', 'often', 'time', 'know'],
 ['azlanti', 'empire', 'aeon', 'star', 'rune', 'drive'],
 ['check', 'area', 'room', 'wall', 'open', 'succeed'],
 ['ship', 'starship', 'rank', 'pilot', 'check', 'crew'],
 ['swarm', 'creatures', 'use', 'creature', 'feet', 'mind'],
 ['station', 'absalom', 'prime', 'ark', 'worlds', 'pact'],
 ['check', 'succeed', 'take', 'one', 'attempt', 'get'],
 ['perception', 'sense', 'init', 'darkvision', 'environment', 'organization'],
 ['damage', 'creature', 'effect', 'target', 'save', 'use'],
 ['fleet', 'pact', 'corpse', 'information', 'worlds', 'know'],
 ['attack', 'drift', 'shield', 'bay', 'forward', 'crew'],
 ['guard', 'see', 'area', 'combat', 'use', 'page'],
 ['room', 'area', 'door', 'wall', 'check', 'skills'],
 ['kac', 'eac', 'fort', 'ref', 'immunities', 'abilities'],
 ['shadow', 'plane', 'planet', 'city', 'world', 'many'],
 ['melee', 'speed', 'abilities', 'combat', 'range', 'critical'],
 ['reptoid', 'reptoids', 'dycepskian', 'chimera', 'algiada'

In [17]:
vect= TfidfVectorizer(analyzer=preprocess_text)
tf_doc_word = vect.fit_transform(data_clean.sect)
tf_doc_word.shape

(6558, 22160)

In [18]:
tf_dtm=pd.DataFrame(tf_doc_word.toarray(),columns=vect.get_feature_names())
tf_dtm.index=df_text.index
tf_dtm

,aanung,abadar,abadaran,abadarans,abadarcorp,abaddon,aballon,aballonian,abandon,abandonment,...,zoologists,zoom,zoos,zunar,zurnala,zusleggim,zyakama,zybollo,zygotes,zysyk
source,,,,,,,,,,,,,,,,,,,,,
Dead_Suns_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dead_Suns_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dead_Suns_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dead_Suns_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dead_Suns_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dead_Suns_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dead_Suns_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dead_Suns_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
tf_nmf_model = NMF(20)
tf_doc_topic = tf_nmf_model.fit_transform(tf_doc_word)
tf_doc_topic.shape

/home/jillian/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


(6558, 20)

In [20]:
tf_topic_word = tf_nmf_model.components_
tf_topic_word.shape

(20, 22160)

In [25]:
tf_words = vect.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-10:-1]
tf_topic_words = [[words[e] for e in l] for l in t]
tf_topic_words

[['computer',
  'computers',
  'data',
  'access',
  'room',
  'check',
  'module',
  'cultists',
  'hack'],
 ['planet',
  'city',
  'worlds',
  'many',
  'pact',
  'one',
  'within',
  'live',
  'world'],
 ['creature',
  'save',
  'effect',
  'target',
  'throw',
  'round',
  'feet',
  'damage',
  'creatures'],
 ['ship',
  'starship',
  'crew',
  'drift',
  'pilot',
  'vessel',
  'rank',
  'captain',
  'engineer'],
 ['check',
  'succeed',
  'successful',
  'attempt',
  'one',
  'take',
  'perception',
  'engineer',
  'bonus'],
 ['swarm',
  'hylax',
  'suskillon',
  'components',
  'mind',
  'shirrens',
  'sdf',
  'hive',
  'units'],
 ['fleet',
  'corpse',
  'eox',
  'waneda',
  'cult',
  'report',
  'bone',
  'eoxian',
  'ministry'],
 ['azlanti',
  'empire',
  'star',
  'aeon',
  'guard',
  'drive',
  'rune',
  'madelon',
  'zolan'],
 ['sun',
  'diver',
  'burn',
  'archipelago',
  'take',
  'fire',
  'khaim',
  'force',
  'bubble'],
 ['area', 'room', 'door', 'wall', 'open', 'doors', 

technology, pactworlds, combat mechanics, starship, game mechanics, bug aliens, eox/undead,
azlanti empire, dawn of flame, area features, ?, horror/signal of screams, ?, factions, combat damage, 

In [22]:
tf_doc_cluster=tf_doc_topic.argmax(axis=1)
pd.Series(tf_doc_cluster).value_counts()

2     1682
13     802
0      489
16     465
1      437
19     310
12     291
15     269
5      265
3      232
17     186
4      175
6      159
7      156
11     130
18     121
10     107
8      105
14      94
9       83
dtype: int64

In [23]:
data_clean2=data_clean.copy()
data_clean2['topic']=doc_topic.argmax(axis=1)

In [24]:
data_clean2.to_pickle('tfid_topics.pkl')
data_clean2

,sect,topic
source,,
Dead_Suns_1,sunrise maiden,3
Dead_Suns_1,tier sanjaval vagabond medium explorer spee...,3
Dead_Suns_1,sunrise maiden map key bridge a pilot s st...,13
Dead_Suns_1,fortune in the stars take your station chec...,1
Dead_Suns_1,the starfinder adventure path is designed to p...,1
...,...,...
Dead_Suns_2,,0
Dead_Suns_2,,0
Dead_Suns_2,bulldoze ex when a yaruk uses its trample a...,14


In [26]:
data_clean2.to_csv('tfid_topics.csv')